### Exploratory Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# read in cleaned data
hs_data = pd.read_csv('processed_nodummies.csv', header=0, sep=',')
hs_data.head()

,School_ID,School_Type,Student_Count_Total,ADA_Accessible,Dress_Code,Bilingual_Services,Refugee_Services,Title_1_Eligible,Average_ACT_School,College_Enrollment_Rate_School,...,Student_Attendance_Change,Teacher_Attendance_Change,One_Year_Dropout_Rate_Change,Student_Low_Income_Pct,Student_Special_Ed_Pct,Student_English_Learners_Pct,Student_Black_Pct,Student_Hispanic_Pct,Student_White_Pct,Student_Asian_Pct
0,609764,Neighborhood,1739,Generally accessible,N,Y,Y,Y,16.5,50.6,...,-0.3,4.8,-0.4,0.953997,0.161587,0.190339,0.033353,0.945371,0.006901,0.002875
1,400054,Charter,992,Generally accessible,Y,N,N,Y,21.9,85.7,...,0.6,0.0,-0.8,0.943548,0.106855,0.120968,0.030242,0.952621,0.009073,0.001008
2,609726,Selective enrollment,959,Fully Accessible,N,N,N,Y,22.5,85.1,...,0.0,4.7,-1.2,0.721585,0.061522,0.002086,0.789364,0.182482,0.007299,0.001043
3,400094,Charter,523,Generally accessible,Y,Y,N,Y,17.5,72.7,...,-0.9,0.0,0.6,0.938815,0.223709,0.137667,0.632887,0.355641,0.003824,0.000000
4,609755,Selective enrollment,2156,Generally accessible,N,Y,N,N,27.9,85.2,...,-0.6,4.7,0.6,0.413265,0.066790,0.010668,0.213822,0.298701,0.282468,0.159091


In [ ]:
# distribution of target variable (graduation rate)

plt.hist(hs_data['Graduation_Rate_School'])
plt.xlabel('Graduation Rate')
plt.ylabel('Frequency')
plt.title('Graduation Rate Distribution')
plt.show()

In [ ]:
# Categorical Variables
hs_data.describe(exclude=[np.number]).T


#### Factors by School Type

In [ ]:
# distribution of school type
hs_data['School_Type'].value_counts().plot(kind='bar', title='School Types')

In [ ]:
#income
hs_data.boxplot(column=['PER CAPITA INCOME '],by=['School_Type'], figsize=(15,8))

In [ ]:
#hardship index
hs_data.boxplot(column=['HARDSHIP INDEX'],by=['School_Type'], figsize=(15,8))

In [ ]:
#Library count by school type
hs_data.boxplot(column=['lib_cnt'],by=['School_Type'], figsize=(15,8))

In [ ]:
# distribution of student growth rating
hs_data['Student_Growth_Rating'].value_counts().plot(kind='bar', title='Student Growth Ratings')

#### School Rating

In [ ]:
#distribution of overall rating
hs_data['Overall_Rating'].value_counts().plot(kind='bar', title='School Rating')

In [ ]:
#rating status
hs_data['Rating_Status'].value_counts().plot(kind='bar', title='Rating Status')

#### Languages

In [ ]:
# distirubtion of overall language
hs_data['PRED_NON_ENG_LANG'].value_counts().plot(kind='bar', title='Non English Languages')

In [ ]:
# Graduation rate by predominant languages
hs_data.boxplot(column=['Graduation_Rate_School'],by=['PRED_NON_ENG_LANG'])

#### Correlation Analysis

In [ ]:
data=pd.read_csv('processed_nodummies.csv', index_col=0)
pd.set_option('display.max_columns', 100)
data.corr()

Graduation rate appears to have a strong positive correlation with College_Enrollment_Rate_School (0.65). Student_Special_Ed_Pct has strong negative correlation with graduation rate (-0.61).

Several variables have a weak positive correlation with graduation rate: 

1. College_Persistence_School_Pct_Year_2 (0.47)
2. Attainment_ACT_Grade_1 (0.45)
3. Growth_ACT_Grade_11_Pct (0.34)
4. Average_ACT_School (0.46)
5. Student_Count_Total (0.39)

#### Target Variable Distribution

In [ ]:
# distribution of target variable (graduation rate)

plt.hist(hs_data['Graduation_Rate_School'])
plt.xlabel('Graduation Rate')
plt.ylabel('Frequency')
plt.title('Graduation Rate Distribution')
plt.show()

In [ ]:
# scatter matrix
pd.plotting.scatter_matrix(data[['Graduation_Rate_School','College_Enrollment_Rate_School','Student_Special_Ed_Pct','College_Persistence_School_Pct_Year_2','Attainment_ACT_Grade_11_Pct','Growth_ACT_Grade_11_Pct','Average_ACT_School','Student_Count_Total']], figsize=(20,20), hist_kwds={'bins':8}, alpha=.5, marker='o', s=50)

Graduation Rate and College Enrollment Rate have a distribution that is left skewed.
Student Special Ed, ACT result variables and Student Count are all skewed to the right.
College Persistence variable is aproximately normally disributed.

### Regression Modeling

#### Random Forest Regressor

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
% matplotlib inline

x=pd.read_csv('processed_scaled.csv',index_col=0)
x.drop('Graduation_Rate_School', axis=1,inplace=True)
y=pd.read_csv('grad_rate.csv', header=None)[1].values

Performing random search to find best hyperparameters

In [ ]:
rf = RandomForestRegressor(criterion='mse', n_jobs=-1, random_state=5)

rf_params = {'n_estimators': randint(5,100), 
             'max_features':randint(10, 50),
             'max_depth': randint(5, 10), 
             'min_samples_split': randint(5, 20)}

rand_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, cv=5, n_jobs=-1, n_iter=50)
rand_search.fit(x, y)
print rand_search.best_score_
print rand_search.best_params_

Building model with best parameters

In [ ]:
# building model with best params
rf = RandomForestRegressor(n_estimators=34, max_features=36, min_samples_split=5, max_depth=8, n_jobs=-1, random_state=5)
rf.fit(x,y)

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_ten = indices[:10]

plt.figure()
plt.title("Feature importances")
plt.bar(range(10), importances[top_ten],
       color="r", align="center")
plt.xticks(range(10), top_ten)
plt.xlim([-1, 10])
plt.show()

In [ ]:
# get names of important features
for i in top_ten:
    print x.columns[i]

### Principal Component Analysis

In [ ]:
continuous_data=data[['Student_Count_Total','Average_ACT_School','College_Enrollment_Rate_School','Growth_ACT_Grade_11_Pct','Attainment_ACT_Grade_11_Pct','Freshmen_On_Track_School_Pct_Year_2','College_Persistence_School_Pct_Year_2','lib_cnt','PRED_NON_ENG_PERCENT','PERCENT OF HOUSING CROWDED','PERCENT HOUSEHOLDS BELOW POVERTY','PERCENT AGED 16+ UNEMPLOYED','PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA','PERCENT AGED UNDER 18 OR OVER 64','PER CAPITA INCOME','HARDSHIP INDEX','Suspensions_Per_100_Students_Change','Misconducts_To_Suspensions_Change','Student_Attendance_Change','Teacher_Attendance_Change','One_Year_Dropout_Rate_Change','Student_Low_Income_Pct','Student_Special_Ed_Pct','Student_English_Learners_Pct','Student_Black_Pct','Student_Hispanic_Pct','Student_White_Pct','Student_Asian_Pct']]
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
min_max.fit(continuous_data)

from sklearn import decomposition
pca =  decomposition.PCA(n_components=10)
data_scaled_pca = pca.fit(continuous_scaled)
np.set_printoptions(precision=2,suppress=True,threshold=np.nan)
pd.set_option('display.max_rows', 500)
print 'Explained Variance per Principal Component'
print(data_scaled_pca.explained_variance_ratio_)

All 10 components explain 91% of the variance. This is nearly 1/3 reduction in variables, while explaining over 90% of the variance.

In [ ]:
pd.DataFrame(data_scaled_pca.components_,columns=continuous_scaled.columns,index = ['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6','PC-7','PC-8','PC-9','PC-10']).T

### Linear Models

### Approach 1: Principal Components + Dummy Variables

All models were run with 5-fold Cross Validation

In [ ]:
from sklearn.linear_model import Lasso, Ridge, SGDRegressor, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV

x=pd.read_csv('pc_dummies.csv',index_col=0)
y=pd.read_csv('grad_rate.csv', header=None)[1].values

##### Lasso

In [ ]:
alphas=np.linspace(.0001,10,20)
param_grid_lasso=dict(alpha=alphas)
lasso=Lasso()
grid_lasso=GridSearchCV(lasso, param_grid_lasso, n_jobs=-1, cv=5,scoring='r2')

result_lasso=grid_lasso.fit(x,y)
print result_lasso.best_params_
print result_lasso.best_score_
print result_lasso.best_estimator_

##### Ridge

In [ ]:
param_grid_ridge=dict(alpha=alphas)
ridge=Ridge()
grid_ridge=GridSearchCV(ridge, param_grid_ridge, n_jobs=-1, cv=5,scoring='r2')

result_ridge=grid_ridge.fit(x,y)
print result_ridge.best_params_
print result_ridge.best_score_
print result_ridge.best_estimator_

##### Elastic Net

In [ ]:
param_grid_EN=dict(alpha=alphas,l1_ratio=l1_ratios)
EN=ElasticNet()
grid_EN=GridSearchCV(EN, param_grid_EN, n_jobs=-1, cv=5,scoring='r2')

result_EN=grid_EN.fit(x,y)
print result_EN.best_params_
print result_EN.best_score_
print result_EN.best_estimator_

Best result out of those 4 models was delivered by Lasso with alpha 0.5264. Elastic Net produced the same result using same alpha and L1_ratio of 1.0, confirming Lasso Regression as the best option. Achieved R-squared is 0.724.

#### Applying Feature Selection

In [ ]:
from sklearn import metrics
from sklearn import feature_selection

from sklearn import cross_validation
linmod = Lasso(alpha=0.5264105263157894)

percentiles = range(1, 100, 1)
results = []
for i in range(1, 100, 1):
    fs = feature_selection.SelectPercentile(feature_selection.f_regression, percentile=i)
    x_fs = fs.fit_transform(x, y)
    scores = cross_validation.cross_val_score(linmod, x_fs, y, cv=5)
    results = np.append(results, scores.mean())

optimal_percentile = percentiles[np.where(results == results.max())[0][0]]
print  "Optimal percentile of features:{0}".format(optimal_percentile), "\n"
optimal_num_features = optimal_percentile*len(x.columns)/100
print "Optimal number of features:{0}".format(optimal_num_features), "\n"

# Plot percentile of features VS. cross-validation scores
import pylab as pl
pl.figure()
pl.xlabel("Percentage of features selected")
pl.ylabel("Cross validation accuracy")
pl.plot(percentiles,results)

The optput says optimal percentile is 32, but improvement is marginal from percentile 9. Choosing less variables and easier interpretability, while sacrificing 0.012 (1.2%) of explained variability.

##### Final Lasso Model with Feature Selection

In [ ]:
fs = feature_selection.SelectPercentile(feature_selection.f_regression, percentile=9)
x_fs = fs.fit_transform(x, y)
linmod.fit(x_fs, y)
names = x.columns.values

%matplotlib inline
fig_size = pl.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 20

def plot_coefficients(model, n_features, feature_names):
    pl.barh(range(n_features), model.coef_, align='center')
    pl.yticks(np.arange(n_features), feature_names)
    pl.xlabel("Coefficient Value")
    pl.ylabel("Feature")
    pl.ylim(-1, n_features)
    pl.rcParams["figure.figsize"] = fig_size
    
plot_coefficients(linmod, len(names[fs.get_support(indices=True)]), names[fs.get_support(indices=True)])

### Approach 2: Full set of variables with/without feature selection

##### Lasso

In [ ]:
lasso = Lasso(max_iter=50000)
l_space = {'alpha': np.logspace(-4,1,20)}
grid = GridSearchCV(estimator=lasso, param_grid = l_space, scoring='neg_mean_squared_error',n_jobs=-1, cv=5)
grid.fit(x, y)

print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

##### Ridge

In [ ]:
ridge = Ridge(max_iter=100000)
r_space = {'alpha': np.logspace(-1,1,20)}
grid = GridSearchCV(estimator=ridge, param_grid = r_space, scoring='neg_mean_squared_error',n_jobs=-1, cv=5)
grid.fit(x, y)

print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

##### Elastic Net

In [ ]:
EN = ElasticNet(max_iter=100000)
en_space = {'alpha': np.logspace(-2,1,20), 'l1_ratio':np.linspace(0.01,1,10)}
grid = GridSearchCV(estimator=EN, param_grid = en_space, scoring='neg_mean_squared_error',n_jobs=-1, cv=5)
grid.fit(x, y)

print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

### With Feature Selection

In [ ]:
from sklearn import metrics
from sklearn import feature_selection
from sklearn.model_selection import cross_val_score

In [ ]:
def get_opt_features(model, x, y):
    percentiles = range(1, 100, 10)
    results = []
    for i in range(1, 100, 10):
        fs = feature_selection.SelectPercentile(feature_selection.f_regression, percentile=i)
        x_fs = fs.fit_transform(x, y)
        scores = cross_val_score(model, x_fs, y, cv=5)
        results = np.append(results, scores.mean())

    optimal_percentile = percentiles[np.where(results == results.max())[0][0]]

    opt_fs = feature_selection.SelectPercentile(feature_selection.f_regression, percentile=optimal_percentile)
    return x_fs, optimal_percentile

##### Lasso with FS

In [ ]:
fs_lasso = Lasso(max_iter=50000)
x_best, p = get_opt_features(fs_lasso, x, y)

grid = GridSearchCV(estimator=fs_lasso, param_grid = l_space, scoring='neg_mean_squared_error',n_jobs=-1, cv=5)
grid.fit(x_best, y)

print 'Feature Percentage: '+str(p)
print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

##### Ridge with FS

In [ ]:
fs_ridge = Ridge()

grid = GridSearchCV(estimator=fs_ridge, param_grid = r_space, scoring='neg_mean_squared_error', cv=5)
grid.fit(x_best, y)

print 'Feature Percentage: '+str(p)
print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

##### Elastic Net with FS

In [ ]:
fs_en = ElasticNet(max_iter=100000)

grid = GridSearchCV(estimator=fs_en, param_grid = en_space, scoring='neg_mean_squared_error',n_jobs=-1, cv=5)
grid.fit(x_best, y)

print 'Feature Percentage: '+str(p)
print 'Best RMSE: '+str(np.sqrt(grid.best_score_*-1))
print grid.best_params_

### Final Model - Lasso

In [4]:
### copy from Olga's Code

## Overall Modeling Summary

#### Approach 1 - PCA and Categorical Features

-  Model Type: Lasso
-  Feature Selection: 16 variables
-  R^2: 0.714

#### Approach 2 - All Features

-  Model Type: Lasso
-  Feature Selection: 60 variables
-  R^2: 0.76

#### General Takeaways:

1. Schools with no data available for student growth rating tend to have lower graduation rate. 
2. Rating status – intensive support has negative impact on school’s graduation rate. 
3. School Type – Citywide option has lower graduation rate.
4. High ACT results correlate to higher graduation rate.
5. Higher college enrollment rates correlate to higher graduation rates.